# Programming for Data Analytics in the Higher Diploma in Science in Data Analytics.

## Project_Superstore

**by Grainne Boyle**

This notebook contains a project that demonstrates what I have learned in this module.

In [2]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
#To ignore warnings, re: the figure layout changes, we import the warnings module.  
import warnings
warnings.filterwarnings('ignore')


In [6]:
# I had difficulty reading in the file, I was getting an error. I used chardet, a library that can detect file encodings.

import chardet  # This imports a library that can detect file encodings

# Opens your file in binary mode ('rb') to read the raw bytes
with open('sample_superstore.csv', 'rb') as file:
    raw_data = file.read()  # Reads the entire file as raw bytes
    result = chardet.detect(raw_data)  # Analyzes the bytes to guess the encoding
    encoding = result['encoding']  # Gets the detected encoding type
    
# Uses the detected encoding to read the CSV file correctly
storedf = pd.read_csv('sample_superstore.csv', encoding=encoding)

In [7]:
# From viewing the data in excel, I decided to remove the following columns as there was either too much detail to analyse, e.g Customer ID or not enough varied detail , e.g Country is only United States.  
storedf = storedf.drop(columns=['Row ID', 'Order ID', 'Customer ID', 'Customer Name', 'Country', 'Postal Code', 'Product ID', 'Product Name'])

In [8]:
print(storedf.head())


   Order Date   Ship Date       Ship Mode    Segment             City  \
0   11/8/2016  11/11/2016    Second Class   Consumer        Henderson   
1   11/8/2016  11/11/2016    Second Class   Consumer        Henderson   
2   6/12/2016   6/16/2016    Second Class  Corporate      Los Angeles   
3  10/11/2015  10/18/2015  Standard Class   Consumer  Fort Lauderdale   
4  10/11/2015  10/18/2015  Standard Class   Consumer  Fort Lauderdale   

        State Region         Category Sub-Category     Sales  Quantity  \
0    Kentucky  South        Furniture    Bookcases  261.9600         2   
1    Kentucky  South        Furniture       Chairs  731.9400         3   
2  California   West  Office Supplies       Labels   14.6200         2   
3     Florida  South        Furniture       Tables  957.5775         5   
4     Florida  South  Office Supplies      Storage   22.3680         2   

   Discount    Profit  
0      0.00   41.9136  
1      0.00  219.5820  
2      0.00    6.8714  
3      0.45 -383.031